In [31]:
import pandas as pd
import requests
import json

# Read the file using pandas
df = pd.read_stata('women2018ok.dta')
#put all city and countries into lists
citylist = df['cityname'].unique().tolist()
countylist= df['countyname'].unique().tolist()
county = df['county'].unique().astype(str).tolist()
county = ['156' + st for st in county]


print(county)


['156110109', '156110111', '156110112', '156110113', '156110114', '156110115', '156110116', '156110117', '156110118', '156110119', '156140105', '156140202', '156140211', '156140225', '156140321', '156140429', '156140802', '156140902', '156140925', '156140926', '156141125', '156141127', '156150105', '156150123', '156150422', '156150521', '156150524', '156150621', '156150625', '156150821', '156150929', '156152201', '156152221', '156152529', '156152921', '156210106', '156210112', '156210321', '156210403', '156210521', '156210681', '156210811', '156211004', '156211224', '156211322', '156211421', '156320205', '156320324', '156320506', '156320582', '156320681', '156320724', '156320812', '156320826', '156320903', '156320922', '156321003', '156321084', '156321112', '156321281', '156321322', '156340124', '156340207', '156340302', '156340311', '156340521', '156340621', '156340803', '156340826', '156341021', '156341204', '156341222', '156341324', '156341503', '156341702', '156370105', '156370113'

In [33]:
kinder_num = {}
for code in county:
    url = "http://api.tianditu.gov.cn/v2/search"
    params = {
        "postStr": '{"keyWord":"幼儿园","queryType":12,"start":0,"count":10,"specify":"'+ code +'","dataTypes":"160101"}',
        "type": "query",
        "tk": "dd54ec94a41e3a323d5cb13eac36c8f1"
    }

    response = requests.get(url, params=params)
    data = response.json()
    kinder_num[code]=data['count']






In [36]:
# Find keys with value 0 in kinder_num
keys_with_zero_value = [key for key, value in kinder_num.items() if value == '0']
# Prompt user to enter corrected values
for key in keys_with_zero_value:
    corrected_value = input(f"Enter corrected value for key {key}: ")
    kinder_num[key] = corrected_value

In [37]:
df['kinder_num'] = df['county'].apply(lambda x: kinder_num['156' + str(x)])


In [9]:
df_county_code = pd.read_csv('county_code.csv')

In [40]:
df.to_csv('women2018_new.csv', index=False)



In [44]:
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].apply(lambda x: x.encode('utf-8') if isinstance(x, str) else x)

df.to_stata('output.dta', version=117, convert_dates={'date_col': 'td'}, write_index=False)

UnicodeEncodeError: 'latin-1' codec can't encode characters in position 0-2: ordinal not in range(256)

In [16]:
# Check for missing values in df_county_code
missing_values = df_county_code[df_county_code['code'].isnull()]

# Prompt the user to fill in the missing values
for index, row in missing_values.iterrows():
    county = row['countyname']
    code = input(f"Please enter the code for {county}: ")
    
    # Update the missing value in df_county_code
    df_county_code.loc[index, 'code'] = code

# Print the updated df_county_code
print(df_county_code)


    countyname    code
0         门头沟区  110109
1          房山区  110111
2          通州区  110112
3          顺义区  110113
4          昌平区  110114
..         ...     ...
321        西山区  530112
322        城关区  540102
323       七里河区  620103
324        西固区  620104
325     嘉峪关市辖区  620300

[326 rows x 2 columns]


In [11]:
df_county_code['code'] = df_county_code['code'].astype(int)
df_county_code = df_county_code.astype(str)

In [18]:
# Filter the county codes that are not 6 digits long
invalid_codes = df_county_code[~df_county_code['code'].str.match(r'^\d{6}$')]

# Prompt the user to check and correct the invalid codes
for index, row in invalid_codes.iterrows():
    county = row['countyname']
    code = row['code']
    code = input(f"Please enter the correct 6-digit code for {county}，{code}: ")
    
    # Update the code in df_county_code
    df_county_code.loc[index, 'code'] = code

# Print the updated df_county_code
print(df_county_code)



    countyname    code
0         门头沟区  110109
1          房山区  110111
2          通州区  110112
3          顺义区  110113
4          昌平区  110114
..         ...     ...
321        西山区  530112
322        城关区  540102
323       七里河区  620103
324        西固区  620104
325     嘉峪关市辖区  620300

[326 rows x 2 columns]


In [19]:
county_code_dict = df_county_code.set_index('countyname')['code'].to_dict()

In [19]:
def getnum(code):
    url = 

print(getnum(county_code_dict['通州区']))


0


In [20]:

# Read the JSON file
data = pd.read_json('All_KDG.json')
data = data['data']['results']
# Convert the JSON data to a DataFrame
all = pd.DataFrame(data)
all = all[['name', 'gbcode']]
all['gbcode'] = all['gbcode'].astype(int)
all = all.astype(str)
all['gbcode'] = all['gbcode'].str[-6:]
all.to_csv('All_KDG.csv', index=False)

In [26]:
def count_rows_by_gbcode(k):
    num = len(all[all['gbcode'] == k])
    return num

county_gbcode_dict = {}

for county in df_county_code['countyname']:
    gbcode = df_county_code[df_county_code['countyname'] == county]['code'].values[0]
    count = count_rows_by_gbcode(gbcode)
    county_gbcode_dict[county] = count

print(county_gbcode_dict)




{'门头沟区': 30, '房山区': 178, '通州区': 336, '顺义区': 131, '昌平区': 280, '大兴区': 219, '怀柔区': 50, '平谷区': 43, '密云区': 53, '延庆县': 37, '小店区': 233, '城区': 0, '南郊区': 8, '浑源县': 13, '平定县': 28, '武乡县': 13, '盐湖区': 186, '忻府区': 37, '宁武县': 12, '静乐县': 17, '柳林县': 54, '岚县': 23, '赛罕区': 136, '和林格尔县': 24, '巴林左旗': 16, '科尔沁左翼中旗': 26, '库伦旗': 8, '达拉特旗': 69, '杭锦旗': 12, '五原县': 13, '四子王旗': 8, '乌兰浩特市': 103, '科尔沁右翼前旗': 33, '正镶白旗': 5, '阿拉善左旗': 16, '铁西区': 329, '东陵区': 102, '台安县': 55, '东洲区': 29, '本溪满族自治县': 12, '东港市': 46, '老边区': 34, '宏伟区': 29, '昌图县': 56, '建平县': 62, '绥中县': 54, '锡山区': 46, '睢宁县': 208, '吴中区': 284, '张家港市': 97, '启东市': 71, '灌南县': 119, '清江浦区': 125, '涟水县': 269, '盐都区': 97, '滨海县': 66, '邗江区': 84, '高邮市': 68, '丹徒区': 28, '兴化市': 104, '沭阳县': 211, '庐江县': 147, '鸠江区': 88, '龙子湖区': 49, '淮上区': 34, '当涂县': 53, '濉溪县': 106, '大观区': 22, '宿松县': 48, '歙县': 25, '颍泉区': 67, '太和县': 122, '泗县': 62, '裕安区': 142, '贵池区': 63, '天桥区': 191, '长清区': 67, '济阳县': 146, '黄岛区': 358, '崂山区': 79, '城阳区': 234, '莱西市': 85, '博山区': 43, '利津县': 34, '坊子区': 83, '任城区': 290, '荣成市': 55

In [27]:
for county, count in county_gbcode_dict.items():
    if count == 0:
        new_gbcode = input(f"Please enter the new gbcode for {county}: ")
        if new_gbcode:
            county_code_dict[county] = new_gbcode

print(county_code_dict)

{'门头沟区': '110109', '房山区': '110111', '通州区': '110112', '顺义区': '110113', '昌平区': '110114', '大兴区': '110115', '怀柔区': '110116', '平谷区': '110117', '密云区': '110118', '延庆县': '110119', '小店区': '140105', '城区': '140202', '南郊区': '140212', '浑源县': '140225', '平定县': '140321', '武乡县': '140429', '盐湖区': '140802', '忻府区': '140902', '宁武县': '140925', '静乐县': '140926', '柳林县': '141125', '岚县': '141127', '赛罕区': '150105', '和林格尔县': '150123', '巴林左旗': '150422', '科尔沁左翼中旗': '150521', '库伦旗': '150524', '达拉特旗': '150621', '杭锦旗': '150625', '五原县': '150821', '四子王旗': '150929', '乌兰浩特市': '152201', '科尔沁右翼前旗': '152221', '正镶白旗': '152529', '阿拉善左旗': '152921', '铁西区': '210106', '东陵区': '210113', '台安县': '210321', '东洲区': '210403', '本溪满族自治县': '210521', '东港市': '210681', '老边区': '210811', '宏伟区': '211004', '昌图县': '211224', '建平县': '211322', '绥中县': '211421', '锡山区': '320205', '睢宁县': '320324', '吴中区': '320506', '张家港市': '320582', '启东市': '320681', '灌南县': '320724', '清江浦区': '320812', '涟水县': '320826', '盐都区': '320903', '滨海县': '320922', '邗江区': '321003', '高邮市': 